# 번호판 인식 학습

OpenCV와 tensorflow를 이용한 번호판 인식 프로그램 입니다.

## 모듈 불러오기

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os

## 이미지 전처리

In [ ]:
image_list = os.listdir('./imgs') # imgs라는 폴더 안의 모든 파일명을 가져옵니다.

roi_list = [] # 실제 프로그램 내에서 사용될 리스트.
labels = [] # 실제 프로그램 내에서 사용될 target값.

for img in image_list:
  roi = cv.imread('./imgs/'+img) # imgs 폴더에 있는 이미지 하나씩 불러옴
  roi_gray = cv.cvtColor(roi,cv.COLOR_BGR2GRAY) # 흑백으로 변환
  #_, thr = cv.threshold(roi_gray, thresh=0.0, maxval=255.0, type=cv.THRESH_BINARY | cv.THRESH_OTSU)# 2진화 수행
  thr = cv.adaptiveThreshold(roi_gray, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 11, 2) # 이진화 시킴. 방식은 apdative threshold
  thr = cv.resize(thr, dsize=(28, 28), interpolation=cv.INTER_AREA) # 28 * 28 으로 이미지 크기 변환
  thr = thr / 255.0 # 0~1 사이의 값으로 정규화
  thr = thr.reshape((28,28,1)) # 딥러닝 모듈에 집어넣기 위해서 2차원 흑백이미지를 3차원으로 변환

  roi_list.append(thr) # 리스트에 추가
  labels.append(int(img[0])) # 파일명의 맨 앞글자(숫자)로 라벨을 지정하고 리스트에 추가

train_images = np.array(roi_list) # 학습될 이미지 numpy array로 변환
train_labels = np.array(labels) # 학습될 라벨 numpy array로 변환

## CNN 모델 생성

In [ ]:
# CNN 모델을 생성한다.

# Convoultion 한 후 Output size 는 다음과 같이 계산된다.
# (W - F + 2P) / S + 1
# W : 원래 이미지 크기
# F : 커널의 크기
# P : 패딩의 크기 (패딩은 이미지 바깥에 임의의 값 영역을 추가로 주는 것을 의미. 보통 0으로 주는 Zero-Padding 방식을 사용)
# S : Stride의 크기 (커널의 이동 거리. 기본값은 1)

# MaxPooling 한 후 Output size 는 다음과 같이 계산된다.
# (W - M) / S + 1
# W : 원래 이미지의 크기
# M : Pooling 커널의 크기
# S : Stride의 크기 (Pooling 커널의 이동 거리. Convoultion과 다르게 Pooling의 기본 Stride값은 2 이다.)

model = models.Sequential() # 모델 생성
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) # 첫번째 층. 입력은 28*28*1의 이미지, 3 * 3 커널 32개를 지니는 Convolution 레이어

                                       # 출력 : (28 - 3 + 0) / 1 + 1 = 26 이고 32개의 커널이 존재하므로 최종적으로 26 * 26 * 32 의 output size를 가진다.

model.add(layers.MaxPooling2D((2, 2))) # 각 26 * 26 에서 2 * 2의 크기를 지니는 커널을 이동시켜가며 가장 큰 값을 뽑아내는 Max Pooling 레이어

                                       # 출력 : (26 - 2) / 2 + 1 = 13 이고 들어온 input이 32개 이므로 최종적으로 13 * 13 * 32 의 output size를 가진다.

model.add(layers.Conv2D(64, (3, 3), activation='relu')) # 64개의 3 * 3 커널을 사용하는 Convolution 레이어

# 출력 : (13 - 3 + 0) / 1 + 1 =  11, 결과 : 11 * 11 * 64

model.add(layers.MaxPooling2D((2, 2)))

# 출력 : (11 - 2) / 2 + 1 = 5.5, 결과 : 5 * 5 * 64

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# 출력 : (5 - 3 + 0) / 1 + 1 = 3, 결과 : 3 * 3 * 64

model.add(layers.Flatten()) # 완전연결신경망으로 데이터를 전달하기위해서 이미지를 1차원 벡터로 펼쳐주는 층. 9 * 9 * 64 = 5184의 1차원 벡터를 생성함.
model.add(layers.Dense(64, activation='relu')) # 64개의 노드를 지니는 일반 층
model.add(layers.Dense(10, activation='softmax')) # 10개의 노드를 지니는 일반층(class 분류 결과를 보여주기위해 softmax 활성화 함수 사용)
model.summary() # 생성된 모델 모양 출력

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

## 모델 컴파일 및 훈련

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
20/20 [==============================] - 30s 13ms/step - loss: 1.8814 - accuracy: 0.4653
Epoch 2/5
20/20 [==============================] - 0s 7ms/step - loss: 0.3888 - accuracy: 0.9164
Epoch 3/5
20/20 [==============================] - 0s 8ms/step - loss: 0.0678 - accuracy: 0.9905
Epoch 4/5
20/20 [==============================] - 0s 7ms/step - loss: 0.0260 - accuracy: 0.9937
Epoch 5/5
20/20 [==============================] - 0s 6ms/step - loss: 0.0171 - accuracy: 0.9984


## 학습한 모델 저장

In [ ]:
model.save('my_cnn_model.h5')